In [24]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from matplotlib import cm
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [25]:
data1=pd.read_csv('phcrime2016.csv')

In [26]:
data1.head(4)

,Unnamed: 0,Dispatch_Date,Dispatch_Time,Location_Block,Lon,Lat,Text_General_Code,year,month,hour
0,1259,2016-03-09,21:26:00,6400 BLOCK MATTHEWS ST,-75.174001,40.056884,All Other Offenses,2016,3,21
1,1419,2016-01-16,00:57:00,5700 BLOCK RISING SUN AV,-75.105683,40.042075,Thefts,2016,1,0
2,1424,2016-01-28,12:46:00,6600 BLOCK MUSGRAVE ST,-75.180695,40.052561,Fraud,2016,1,12
3,1426,2016-01-26,15:52:00,4500 BLOCK N 16TH ST,-75.151526,40.022889,Fraud,2016,1,15


In [27]:
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dispatch_Date'])
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Date'].dt.day
    data['DayOfWeek'] = data['Date'].dt.weekday
    data['Month'] = data['Date'].dt.month
    data['Year'] = data['Date'].dt.year
    return data
data2 = feature_engineering(data1)

In [28]:
data2

,Unnamed: 0,Dispatch_Date,Dispatch_Time,Location_Block,Lon,Lat,Text_General_Code,year,month,hour,Date,n_days,Day,DayOfWeek,Month,Year
0,1259,2016-03-09,21:26:00,6400 BLOCK MATTHEWS ST,-75.174001,40.056884,All Other Offenses,2016,3,21,2016-03-09,68,9,2,3,2016
1,1419,2016-01-16,00:57:00,5700 BLOCK RISING SUN AV,-75.105683,40.042075,Thefts,2016,1,0,2016-01-16,15,16,5,1,2016
2,1424,2016-01-28,12:46:00,6600 BLOCK MUSGRAVE ST,-75.180695,40.052561,Fraud,2016,1,12,2016-01-28,27,28,3,1,2016
3,1426,2016-01-26,15:52:00,4500 BLOCK N 16TH ST,-75.151526,40.022889,Fraud,2016,1,15,2016-01-26,25,26,1,1,2016
4,1429,2016-01-31,16:03:00,700 BLOCK ARCH ST,-75.151599,39.952923,Rape,2016,1,16,2016-01-31,30,31,6,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165613,2237414,2016-12-28,20:24:00,200 BLOCK W LEHIGH AVE,-75.134880,39.991461,Narcotic / Drug Law Violations,2016,12,20,2016-12-28,362,28,2,12,2016
165614,2237415,2016-12-28,15:54:00,5300 BLOCK GAINOR RD,-75.230457,39.989421,Fraud,2016,12,15,2016-12-28,362,28,2,12,2016
165615,2237416,2016-12-28,18:48:00,0 BLOCK PIA WAY,-75.232354,39.883546,Fraud,2016,12,18,2016-12-28,362,28,2,12,2016
165616,2237417,2016-12-28,10:14:00,1600 BLOCK MARKET ST,-75.168407,39.952938,Forgery and Counterfeiting,2016,12,10,2016-12-28,362,28,2,12,2016


In [14]:
data2.drop(columns=['year','Dispatch_Date_Time','Dispatch_Time'], inplace=True)

In [15]:
data2.drop(columns=['Dc_Dist'], inplace=True)

In [8]:
data2.drop(columns=['hour','min'], inplace=True)

In [16]:
data2.drop(columns=['Dispatch_Date'], inplace=True)

In [18]:
data2.drop(columns=['Psa','Date'], inplace=True)

In [20]:
data2.drop(columns=['month','Year'], inplace=True)

In [17]:
data2.head(4)

,Psa,Location_Block,Text_General_Code,Police_Districts,Lon,Lat,month,hour,min,day_time,Date,n_days,Day,DayOfWeek,Month,Year
0,1,800 BLOCK S BROAD ST,Thefts,13.0,-75.166412,39.940070,4,12,50,12.83,2015-04-25,114,25,5,4,2015
1,3,1500 BLOCK N 15TH ST,Thefts,16.0,-75.160229,39.976134,10,18,18,18.30,2015-10-06,278,6,1,10,2015
2,3,1500 BLOCK N 17TH ST,Thefts,16.0,-75.163073,39.977872,10,0,49,0.82,2015-10-09,281,9,4,10,2015
3,1,6500 BLOCK CASTOR AV,Thefts,2.0,-75.075602,40.041270,11,8,39,8.65,2015-11-30,333,30,0,11,2015


In [29]:
data1['Text_General_Code'].value_counts()

All Other Offenses                         29052
Other Assaults                             23519
Thefts                                     22326
Vandalism/Criminal Mischief                15492
Theft from Vehicle                         13651
Fraud                                      11656
Narcotic / Drug Law Violations              7805
Recovered Stolen Motor Vehicle              7585
Burglary Residential                        5494
Aggravated Assault No Firearm               5248
DRIVING UNDER THE INFLUENCE                 3685
Robbery No Firearm                          3405
Robbery Firearm                             2634
Aggravated Assault Firearm                  2214
Motor Vehicle Theft                         2112
Weapon Violations                           1878
Disorderly Conduct                          1593
Burglary Non-Residential                    1377
Rape                                        1155
Prostitution and Commercialized Vice         903
Other Sex Offenses (

In [30]:
import numpy as np
THEFT = ["Thefts", "Theft from Vehicle","Burglary Residential","Motor Vehicle Theft","Burglary Non-Residential"]
IMPAIRED = ["Narcotic / Drug Law Violations","DRIVING UNDER THE INFLUENCE","Public Drunkenness","Liquor Law Violations"]
VIOLENCE = ["Aggravated Assault No Firearm","Robbery Firearm", "Rape","Other Sex Offenses (Not Commercialized)", 
"Arson", "Homicide - Criminal"]
OTHER = ["All Other Offenses"]

selections = [(data1['Text_General_Code'].isin(THEFT)),
(data1['Text_General_Code'].isin(IMPAIRED)),
(data1['Text_General_Code'].isin(VIOLENCE)),
(data1['Text_General_Code'].isin(OTHER))]
new_categories = ['THEFT', 'IMPAIRED', 'VIOLENCE', 'OTHER']
data1['CAT'] = np.select(selections, new_categories, default='NaN')

In [31]:
data1['CAT'].value_counts()

NaN         69035
THEFT       44960
OTHER       29052
IMPAIRED    11988
VIOLENCE    10583
Name: CAT, dtype: int64

In [13]:
#重分类
import numpy as np
THEFT = ["LARCENY/THEFT", "VEHICLE THEFT", "BURGLARY", "ROBBERY", "STOLEN PROPERTY"]
IMPAIRED = ["DRUNKENNESS", "DRIVING UNDER THE INFLUENCE", "LIQUOR LAWS", 
"DISORDERLY CONDUCT", "DRUG/NARCOTIC", "LOITERING"]
VIOLENCE = ["ASSAULT", "VANDALISM", "SUSPICIOUS OCC", "TRESPASS", 
"SEX OFFENSES,FORCIBLE" , "SEX OFFENSES, NON FORCIBLE"]
OTHER = ["OTHER OFFENSES", "NON-CRIMINAL"]

selections = [(data1['Category'].isin(THEFT)),
(data1['Category'].isin(IMPAIRED)),
(data1['Category'].isin(VIOLENCE)),
(data1['Category'].isin(OTHER))]
new_categories = ['THEFT', 'IMPAIRED', 'VIOLENCE', 'OTHER']
data1['CAT'] = np.select(selections, new_categories, default='NaN')

In [32]:
data2=data1.drop(data1[(data1.CAT=='NaN')].index.tolist())
data2['CAT'].value_counts()

THEFT       44960
OTHER       29052
IMPAIRED    11988
VIOLENCE    10583
Name: CAT, dtype: int64

In [33]:
data2.drop(columns=['Text_General_Code'], inplace=True)

In [34]:
data2

,Unnamed: 0,Dispatch_Date,Dispatch_Time,Location_Block,Lon,Lat,year,month,hour,Date,n_days,Day,DayOfWeek,Month,Year,CAT
0,1259,2016-03-09,21:26:00,6400 BLOCK MATTHEWS ST,-75.174001,40.056884,2016,3,21,2016-03-09,68,9,2,3,2016,OTHER
1,1419,2016-01-16,00:57:00,5700 BLOCK RISING SUN AV,-75.105683,40.042075,2016,1,0,2016-01-16,15,16,5,1,2016,THEFT
4,1429,2016-01-31,16:03:00,700 BLOCK ARCH ST,-75.151599,39.952923,2016,1,16,2016-01-31,30,31,6,1,2016,VIOLENCE
5,1433,2016-02-09,11:09:00,100 BLOCK N 33RD ST,-75.189048,39.957960,2016,2,11,2016-02-09,39,9,1,2,2016,THEFT
6,1438,2016-01-16,02:05:00,0 BLOCK PIA WAY,-75.230706,39.883852,2016,1,2,2016-01-16,15,16,5,1,2016,THEFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165610,2237396,2016-12-28,23:27:00,600 BLOCK N 55TH ST,-75.230611,39.970898,2016,12,23,2016-12-28,362,28,2,12,2016,VIOLENCE
165611,2237397,2016-12-28,18:57:00,300 BLOCK EARLHAM TER,-75.176843,40.027777,2016,12,18,2016-12-28,362,28,2,12,2016,VIOLENCE
165612,2237398,2016-12-28,19:46:00,300 BLOCK EARLHAM TER,-75.176843,40.027777,2016,12,19,2016-12-28,362,28,2,12,2016,VIOLENCE
165613,2237414,2016-12-28,20:24:00,200 BLOCK W LEHIGH AVE,-75.134880,39.991461,2016,12,20,2016-12-28,362,28,2,12,2016,IMPAIRED


In [35]:
data2.to_csv('phnew_crime2016.csv')